In [2]:
import mysql.connector
from textblob import TextBlob
from scipy.spatial.distance import cosine
from TrainTest import TrainTest
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances
import csv
import scipy.sparse as sp
from scipy.sparse.linalg import svds
pd.options.mode.chained_assignment = None


In [3]:
#connect to mysql
cnx = mysql.connector.connect(host='152.19.68.141', user='ctolson', password='ilaYOU5!', database='sephora_cosmetics')
cursor = cnx.cursor()

#query reviews by product1
query = ("SELECT R.product_id, review_id, review, reviewer, type "
         "FROM Reviews as R "
         "JOIN Product as P "
         "ON P.product_id = R.product_id ")
cursor.execute(query)


#close mysql server
cnx.close()

In [4]:
#clean reviews data
product_id = []
review_id = []
reviews = []
reviewers = []
types = []
for (x, y, z, w, v) in cursor:
    product_id.append(int(x))
    review_id.append(int(y))
    reviews.append(z)
    reviewers.append(w)
    types.append(v)

#convert to data frame
data = list(zip(product_id, review_id, reviews, reviewers, types))
review_data = pd.DataFrame(data=data, index=range(0,len(reviews)), columns=['product_id', 'review_id', 'reviews', 'reviewers', 'types'])


In [5]:
#get training set review ids
training = TrainTest()
ttset = training.getSet()
train = ttset['train']

#create training set
review_train = review_data.loc[review_data['review_id'].isin(train)]

#convert series to lists
reviews = review_train['reviews'].tolist()


In [ ]:
#tokenizer function
def stemTokenizer(text):
    #set stop words 
    stop_words = set(stopwords.words('english'))
    stop_words.update(['.', ',', '"', "'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}']) # remove it if you need punctuation 
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stop_words]
    
    # filter out any tokens not containing letters (e.g., numeric tokens, raw punctuation)
    filtered_tokens = [token for token in tokens if re.search('[a-zA-Z]', token)]     
            
    #stem tokens
    sb_stemmer = SnowballStemmer("english")
    stemmed_tokens = [sb_stemmer.stem(i) for i in filtered_tokens]
    
    return stemmed_tokens

#get tokens
stemmed_tokens = [stemTokenizer(x) for x in reviews]


In [6]:
#get sentiment polarity for reviews
sentiment = [TextBlob(x).sentiment.polarity for x in reviews]
sentiment = [int(x>0) for x in sentiment]
sentiment = pd.Series(sentiment, index=review_train.index)


In [7]:
#create user-item matrix
review_train['sentiment'] = sentiment
review_pivot = pd.pivot_table(review_train, index='product_id',columns='reviewers',values='sentiment',fill_value=0)

#open file 
fl = open('../../Data/dist_reviews_item.csv', 'w')
writer = csv.writer(fl)

#get pairwise distances
dist = pairwise_distances(review_pivot, metric='jaccard')

#save pairwise distances
for i in dist:
    writer.writerow(i)

#close file
fl.close()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype float64 was converted to bool by check_pairwise_arrays.
  warnings.warn(msg, _DataConversionWarning)
